# JSON 필드
- JSON 필드는 벡터 임베딩과 함께 추가 정보를 키-값 쌍으로 저장하는 스칼라 필드

## 제한
- 필드 크기: JSON 필드 크기는 65,536바이트로 제한
- 중첩된 사전: JSON 필드 값 내에 중첩된 dictionary는 저장 시 일반 문자열로 취급
- 기본값: JSON 필드는 기본값 x 그러나 nullable 속성을 True로 설정하여 Null값 허용
- 유형 일치: JSON 필드의 키 값이 정수 또는 실수인 경우 표현식 필터를 통해서만 같은 유형의 다른 숫자 키와 비교할 수 있음
- 이름 지정: JSON 키의 이름을 지정할 때는 문자, 숫자, 밑줄만 사용하는 것이 좋음 -> 다른 문자 사용시 필터링이나 검색시 문제 생김
- 문자열 처리: 'a"b', "a'b", 'a\\'b', "a\\"b" 는 그대로 저장, 'a'b' 와 "a"b" 는 유효하지 않은 것으로 간주
- JSON 인덱싱: JSON 필드를 색인할 때 JSON 필드에 하나 이상의 경로를 지정하여 필터링을 가속화할 수 있음 

- 참고자료: https://milvus.io/docs/ko/use-json-fields.md

## JSON 필드 추가
- 컬렉션 스키마에 이 JSON 필드 metadata를 추가하려면 DataType.JSON을 사용

In [1]:
from pymilvus import MilvusClient, DataType

SERVER_ADDR = "http://localhost:19530"

client = MilvusClient(uri=SERVER_ADDR)

schema = client.create_schema(
    auto_id=False,
    enable_dynamic_fields=True,
)

schema.add_field(field_name="metadata", datatype=DataType.JSON, nullable=True)
schema.add_field(field_name="pk", datatype=DataType.INT64, is_primary=True)
schema.add_field(field_name="embedding", datatype=DataType.FLOAT_VECTOR, dim=3)

{'auto_id': False, 'description': '', 'fields': [{'name': 'metadata', 'description': '', 'type': <DataType.JSON: 23>, 'nullable': True}, {'name': 'pk', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 3}}], 'enable_dynamic_field': False}

## 인덱스 매개변수 설정
- 벡터 필드의 경우 필수
- JSON 필드의 경우 선택 사항

### JSON 필드 색인화
- 기본적으로 JSON 필드는 색인화되지 않으므로 모든 필터 쿼리는 모든 행을 스캔해야 함
- field_name: 스키마에 있는 JSON 필드의 이름
- index_type: 생성할 인덱스 유형 JSON 경로 인덱싱에는 INVERTED만 지원됨

In [ ]:
index_params = client.prepare_index_params()

# Example 1: Index the 'category' key inside 'product_info' as a string
index_params.add_index(
    field_name="metadata", # JSON field name to index
    index_type="INVERTED", # Index type. Set to INVERTED
    index_name="json_index_1", # Index name
    params={
        "json_path": "metadata[\"product_info\"][\"category\"]", # Path in JSON field to index
        "json_cast_type": "varchar" # Data type that the extracted JSON values will be cast to
    }
)

# Example 2: Index 'price' as a numeric type (double)
index_params.add_index(
    field_name="metadata",
    index_type="INVERTED",
    index_name="json_index_2",
    params={
        "json_path": "metadata[\"price\"]",
        "json_cast_type": "double"
    }
)
